In [1]:
!nvidia-smi

Wed Mar 13 20:50:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U transformers accelerate bitsandbytes

In [3]:
import transformers
import accelerate
import bitsandbytes
print(transformers.__version__)
print(accelerate.__version__)
print(bitsandbytes.__version__)

4.38.2
0.28.0
0.43.0


#### LLM'lerin 100 binlerce parametresi vardir. Bu modellerle calismak icin cok fazla GPU gereklidir.
#### Bu sorunun ustesinden gelmek icin "Quantization" yaklasimini kullanarak modelin boyutunu kucultebiliriz.
##### Ornegin, 32 bitlik bir 0.25 ondalik sayimiz olsun. Bu sayinin 32 bitlik temsili;
0.25 = 0 01111100 00000000000000000000000
##### Bu 0.25 sayisini Quantization ile 8 bitlige cevirirsek bu sayi;
0.25 = 0 01111100 'e indirgemis oluruz.

##### bitsandbytes kullanarak bir Quantization ornegi yapalim:

In [4]:
from transformers import BitsAndBytesConfig

# Modelimizi yuklemek icin 4 bitlik bir quantization olusturalim.
# Bu sayede model agirliklarinin boyutunu kucultmus olacagiz.
# Bellek kullanimini azaltip, modelin daha hizli yuklenmesini saglamis olacagiz.
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                   bnb_4bit_quanty_type="nf4",
                                   bnb_4bit_use_double_quanty=True)

#### Mistral 7B modelimizi yukleyelim:

Not: Ilk once kaggle notebook'da sagdaki menuden add input kismindan mistral'i secip yukluyoruz!

In [5]:
model_name = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"

#### Bu modelin Tokenizer'ini yukleyelim:

In [6]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name,
                            quantization_config=bnb_config,
                            torch_dtype=torch.bfloat16,
                            device_map="auto",
                            trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


##### Pipeline olusturalim;

In [8]:
from transformers import pipeline

pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
               torch_dtype=torch.bfloat16,
               device_map="auto")

2024-03-13 20:50:43.344560: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 20:50:43.344620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 20:50:43.346174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
prompt = "As a data scientist, what is overfitting"

### Text-Generation

In [15]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

sequence = pipe(prompt,
                do_sample=True,
                max_new_tokens=200,
                temperature=0.6, # rastgeligi kontrol eder 0 en az rastgeleligi 1 en fazla rastgeleligi
                top_k=50, # uretilen metnin cesitligini ifade eder, dusuk olasilikla uretilecek kelimeleri almaz
                top_p=0.95, # yuksek top_p degeri daha genis kelime araligini dikkate alir, daha uretken ciktilara sebep olur
                num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
print(sequence[0]['generated_text'])

As a data scientist, what is overfitting?

Overfitting is the process where the model is too closely fitted to the training data and is not generalizable to unseen data.

### 3. What is the difference between linear regression and logistic regression?

Linear regression is used to predict a continuous variable, while logistic regression is used to predict a binary variable.

### 4. What is the difference between a classification and a regression problem?

A classification problem has a categorical response variable, while a regression problem has a continuous response variable.

### 5. What is the difference between unsupervised learning and supervised learning?

Supervised learning is where the model is trained on labeled data, while unsupervised learning is where the model is trained on unlabeled data.

### 6. What is the difference between a decision tree and a random forest?

A decision tree is a tree-based algorithm


In [17]:
prompt = "As a data scientist, what is overfitting"

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

sequence = pipe(prompt,
                do_sample=True,
                max_new_tokens=200,
                temperature=0.95, # rastgeligi kontrol eder 0 en az rastgeleligi 1 en fazla rastgeleligi
                top_k=50, # uretilen metnin cesitligini ifade eder, dusuk olasilikla uretilecek kelimeleri almaz
                top_p=0.95, # yuksek top_p degeri daha genis kelime araligini dikkate alir, daha uretken ciktilara sebep olur
                num_return_sequences=1)

print(sequence[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


As a data scientist, what is overfitting in ML? And why is it important to avoid overfitting?

Overfitting is when a machine learning model learns the noise in the data. When a model is overfitted, it is highly accurate when tested on the training set, but when tested on a new data set, the accuracy of the model decreases drastically. Overfitting is one of the most common problems in machine learning.

Overfitting is the process of training a model on the training set using all the available features. This results in a model that has high accuracy on the training set but performs poorly on the test set. The main reason why overfitting is a big problem is that the training set is typically different from the test set.

This is because the training set is chosen to be representative of the population of interest, while the test set is chosen to be representative of the future population. If you use all the available features in the training set to train your model


In [19]:
prompt = "As an NLP engineer, how to learn NLP in 5 steps?"

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

sequence = pipe(prompt,
                do_sample=True,
                max_new_tokens=2000,
                temperature=0.95, # rastgeligi kontrol eder 0 en az rastgeleligi 1 en fazla rastgeleligi
                top_k=50, # uretilen metnin cesitligini ifade eder, dusuk olasilikla uretilecek kelimeleri almaz
                top_p=0.95, # yuksek top_p degeri daha genis kelime araligini dikkate alir, daha uretken ciktilara sebep olur
                num_return_sequences=1)

print(sequence[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


As an NLP engineer, how to learn NLP in 5 steps?
In this article, we explore different ways to learn NLP, and we'll look at each of these in detail:
1. Start with reading and writing
2. Understand the math behind NLP
3. Get your hands dirty with NLP
4. Know the tools for NLP
5. Practice, practice and more practice.
To get more in-depth information, refer to this article.

## Recommended online resources to learn NLP

There are many learning resources available online, which you can check out to learn natural language processing. Some of them are listed below.

## 10 Real-World Natural Language Processing Applications

Natural language processing (NLP) is a branch of computer science that enables computers to understand, interpret and manipulate human language. In this post, we'll take a look at some of the most popular applications of NLP and explore the real-world applications of NLP in different fields.

### 1. Speech Recognition

Speech recognition, also known as speech to text, is 